In [ ]:
!nvidia-smi

In [ ]:
%pip install rich==13.0.1
%pip install -q gdown inference-gpu --upgrade
%pip uninstall -y pymc
%pip install pymc==5.12.0
%pip install -q onnxruntime-gpu==1.18.0 --index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/

In [ ]:
%pip install --no-cache-dir package_name
%pip install -q git+https://github.com/roboflow/sports.git

In [ ]:
%pip install -q supervision>=0.23.0

In [ ]:
%pip install --no-cache-dir

In [ ]:
%pip install --upgrade pip

In [ ]:
%pip install numpy==1.26.0
%pip install inference-gpu

In [ ]:
%pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org inference


In [ ]:
%pip install "C:/Users/user/Downloads/inference-0.43.0-py3-none-any.whl"

In [ ]:
%pip install inference

In [ ]:
from inference import get_model
#from google.colab import userdata

ROBOFLOW_API_KEY = "ITfUpuY5QO9WTBpcEXTh"  # Replace with your actual Roboflow API key

PLAYER_DETECTION_MODEL_ID = "football-players-detection-3zvbc/12"
PLAYER_DETECTION_MODEL = get_model(model_id=PLAYER_DETECTION_MODEL_ID, api_key=ROBOFLOW_API_KEY)

Simple annotation with Boxes

In [ ]:
SOURCE_VIDEO_PATH = "C:/Users/user/Desktop/FootballAI/videos/goalMatch.mp4"


In [ ]:
import supervision as sv

box_annotator = sv.BoxAnnotator(
    color=sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000')
)


In [ ]:

frame_number = 0  # Change to the desired frame number
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

for i, frame in enumerate(frame_generator):
    if i == frame_number:
        # Run inference on the specific frame
        result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
        detections = sv.Detections.from_inference(result)

        # Annotate the frame
        annotated_frame = frame.copy()
        annotated_frame = box_annotator.annotate(
            scene=annotated_frame,
            detections=detections
        )
        
        labels = [
          f"{class_name} {confidence:.2f}"
          for class_name, confidence
          in zip(detections["class_name"] , detections.confidence)
         ]
        annotated_frame = label_annotator.annotate(annotated_frame , detections,labels =labels)##annotate the frame with the labels

         # Show the annotated frame
        sv.plot_image(annotated_frame)
        break  # Stop after processing the specified frame


GENERATING VIDEO WITH ANNOTATIONS

In [ ]:
from tqdm import tqdm
import supervision as sv

SOURCE_VIDEO_PATH = "C:/Users/user/Desktop/FootballAI/121364_0.mp4"
TARGET_VIDEO_PATH = "C:/Users/user/Desktop/FootballAI/121364_0_result.mp4"

box_annotator = sv.BoxAnnotator(##Box annotator options like coloring and thickness
    color = sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
    thickness = 2
)


In [ ]:
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)##I will take the info from the source video such as resolution and frame rate to pass into the result video 
video_sink = sv.VideoSink(TARGET_VIDEO_PATH, video_info=video_info)##The result video
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)


In [ ]:



with video_sink:
     for frame in tqdm(frame_generator , total = video_info.total_frames):



      label_annotator = sv.LabelAnnotator(##labeling the box and adding text
          color = sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
          text_color = sv.Color.from_hex('#000000')
      )

      frame = next(frame_generator)

      result = PLAYER_DETECTION_MODEL.infer(frame,confidence = 0.3)[0]##This give me a row data of the detection
      detections = sv.Detections.from_inference(result)##Converting the row data into standarized format


      labels = [
          f"{class_name} {confidence:.2f}"
          for class_name, confidence
          in zip(detections["class_name"] , detections.confidence)
      ]

      annotated_frame = frame.copy()##Copying the frame to anotate it with boxes
      annotated_frame = box_annotator.annotate(annotated_frame , detections)##annotate the frame with the detections
      annotated_frame = label_annotator.annotate(annotated_frame , detections,labels =labels)##annotate the frame with the labels
      video_sink.write_frame(annotated_frame)##writing the annotated frame and looping over all video frame 

In [ ]:
def assign_ball_to_player(players, ball_bbox):
    """
    Assigns the ball to the closest player based on bounding box center distance.
    Returns the index of the player controlling the ball, or -1 if no player is close enough.
    """
    ball_center = np.array([(ball_bbox[0] + ball_bbox[2]) / 2, (ball_bbox[1] + ball_bbox[3]) / 2])
    min_distance = float('inf')
    assigned_player = -1
    
    for i, player in enumerate(players):
        player_bbox = player['bounding_box']
        player_center = np.array([(player_bbox[0] + player_bbox[2]) / 2, (player_bbox[1] + player_bbox[3]) / 2])
        distance = np.linalg.norm(player_center - ball_center)
        
        if distance < min_distance:
            min_distance = distance
            assigned_player = i
    
    return assigned_player

In [ ]:
%pip install ultralytics


In [38]:
import supervision as sv
from TeamClassifier import TeamClassifier

SOURCE_VIDEO_PATH = "C:/Users/user/Desktop/FootballAI/121364_0.mp4"

BALL_ID = 0
PLAYER_ID = 2

crops = extract_crops(SOURCE_VIDEO_PATH)

team_classifier = TeamClassifier(device=DEVICE)
team_classifier.fit(crops)

ellipse_annotator = sv.EllipseAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000'),
    text_position=sv.Position.BOTTOM_CENTER
)
triangle_annotator = sv.TriangleAnnotator(
    color=sv.Color.from_hex('#FFD700'),
    base=25,
    height=21,
    outline_thickness=1
)

tracker = sv.ByteTrack()
tracker.reset()

Extracting crops: 0it [00:06, ?it/s]


KeyboardInterrupt: 

In [36]:

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
detections = sv.Detections.from_inference(result)

ball_detections = detections[detections.class_id == BALL_ID]
ball_detections.xyxy = sv.pad_boxes(xyxy=ball_detections.xyxy, px=10)

all_detections = detections[detections.class_id != BALL_ID]
all_detections = all_detections.with_nms(threshold=0.5, class_agnostic=True)
all_detections = tracker.update_with_detections(detections=all_detections)

players_detections = all_detections[all_detections.class_id == PLAYER_ID]
players_crops = [sv.crop_image(frame,xyxy) for xyxy in players_detections.xyxy]
players_detections.class_id = team_classifier.predict(players_crops)

labels = [
    f"#{tracker_id}"
    for tracker_id
    in players_detections.tracker_id
]

annotated_frame = frame.copy()
annotated_frame = ellipse_annotator.annotate( scene=annotated_frame, detections = players_detections)
annotated_frame = label_annotator.annotate(scene=annotated_frame,detections = players_detections, labels=labels)
annotated_frame = triangle_annotator.annotate( scene=annotated_frame, detections = ball_detections)

sv.plot_image(annotated_frame)

NameError: name 'tracker' is not defined

In [ ]:
CLASSIFY THE TEAMS

In [ ]:
from tqdm import tqdm 
SOURCE_VIDEO_PATH = "C:/Users/user/Desktop/FootballAI/121364_0.mp4"

STRIDE = 30 
PLAYER_ID =2
def extract_crops(source_video_path : str):
        frame_generator = sv.get_video_frames_generator(source_video_path, stride=STRIDE)##Getting the frames with a stride of 30

        crops = []

        for frame in tqdm(frame_generator , desc="Extracting crops"):
            result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.5)[0]
            detections = sv.Detections.from_inference(result)
            detections = detections.with_nms(threshold=0.5 , class_agnostic=True)
            detections = detections[detections.class_id == PLAYER_ID]

            crops += [
                sv.crop_image(frame,xyxy)
                for xyxy
                in detections.xyxy
            ]
        return crops

In [ ]:
crops = extract_crops(SOURCE_VIDEO_PATH)

In [ ]:
import supervision as sv

sv.plot_images_grid(crops[:100] , grid_size=(10,10) )

In [ ]:
#%pip install torch
#%pip install transformers
%pip install sentencepiece
import sentencepiece
import torch
from transformers import AutoProcessor , SiglipVisionModel

SIGLIP_MODEL_PATH = 'google/siglip-base-patch16-224'

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

EMBEDDING_MODEL = SiglipVisionModel.from_pretrained(SIGLIP_MODEL_PATH).to(DEVICE)
EMBEDDING_PROCESSOR = AutoProcessor.from_pretrained(SIGLIP_MODEL_PATH)

In [ ]:

print(f"Number of valid crops: {len(crops)}")

In [ ]:
%pip install more-itertools
from more_itertools import chunked
import numpy as np
BATCH_SIZE = 32

crops = [sv.cv2_to_pillow(crop) for crop in crops]
batches = chunked(crops , BATCH_SIZE)

data = []
with torch.no_grad():
        for batch in tqdm(batches , desc = 'Extracting embeddings'):
            inputs = EMBEDDING_PROCESSOR(images = batch , return_tensors = 'pt').to(DEVICE)
            output = EMBEDDING_MODEL(**inputs)

            embeddings = torch.mean(output.last_hidden_state , dim = 1).cpu().numpy()
            data.append(embeddings)
data = np.concatenate(data)

In [ ]:
data.shape
data

In [ ]:
%pip install umap-learn
import umap
from sklearn.cluster import KMeans

REDUCER = umap.UMAP(n_components=3)
CLUSTERING_MODEL = KMeans(n_clusters=2)

In [ ]:
projections = REDUCER.fit_transform(data)

In [ ]:
projections.shape

In [ ]:
clusters = CLUSTERING_MODEL.fit_predict(projections)